In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import folium

In [2]:
tods = [1,2,3,4,5,6,7,8]

In [3]:
df_all = pd.DataFrame()
conflation = pd.read_csv('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Community Areas/Community_Area_to_Census_Tract.csv')
conflation['COMMUNITY_AREA'] = conflation.area_num_1.astype(float)
for tod in tods:
    df3 = pd.DataFrame()

    print('Working on tod ' + str(tod))
    df = pd.read_hdf('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Old Files/Chicago_TNC_Trips_20_Incomplete.H5', key = 'Weekday_' + str(tod))
    
    df = df[df['MONTH'].isin([1,2])]
    df = df[df['YEAR']==2019]
    df['DAY'] = df['Trip Start Timestamp'].dt.day
    
    print(str(len(df[(~np.isnan(df['Pickup Community Area']))&(np.isnan(df['Pickup Census Tract']))&(~np.isnan(df['Dropoff Community Area'])) & (np.isnan(df['Dropoff Census Tract']))])) +  ' Trip Records with Suppressed Origin and Destination out of ' + str(len(df)) )
    print(str(len(df[((np.isnan(df['Pickup Community Area']))&(~np.isnan(df['Pickup Census Tract'])))|((np.isnan(df['Dropoff Community Area'])) & (np.isnan(df['Dropoff Census Tract'])))])) +  ' Trip Records Outside of Chicago but within cook county out of ' + str(len(df)) )
    print(str(len(df[((~np.isnan(df['Pickup Community Area']))&(~np.isnan(df['Pickup Census Tract'])))&((~np.isnan(df['Dropoff Community Area'])) & (~np.isnan(df['Dropoff Census Tract'])))])) +  ' Trip Records with both trip ends within Chicago out of ' + str(len(df)) )
    print(str(len(df[(np.isnan(df['Pickup Community Area']))|(np.isnan(df['Dropoff Community Area']))])) +  ' Trip Records with one trip end outside of Cook county out of ' + str(len(df)) )

    
    #select out the trips that have community area data and are missing census tract data 
    df['SUP_PRIVATE_TRIPS'] = np.where(df['Shared Trip Authorized'] == False, 1, 0)
    df['SUP_SHARED_TRIPS'] = np.where(df['Shared Trip Authorized'] == True, 1, 0)
    df = df[(np.isnan(df['Pickup Census Tract']))|(np.isnan(df['Dropoff Census Tract']))]
    df = df[~np.isnan(df['Pickup Community Area'])&(~np.isnan(df['Dropoff Community Area']))]
    
    
    df = df.groupby(by = ['Pickup Community Area', 'Dropoff Community Area', 'MONTH','DAY'], as_index = False).sum()
    df = df.groupby(by = ['Pickup Community Area', 'Dropoff Community Area'], as_index = False).mean()

    print('There are ' +str(df.SUP_PRIVATE_TRIPS.sum() + df.SUP_SHARED_TRIPS.sum()) + ' Average Weekday Trips that are Suppressed!')
    
    #make a column to iterate through
    df['OD_PAIRS'] = df['Pickup Community Area'].astype(str) + '_' + df['Dropoff Community Area'].astype(str)
    
    #iterate through each of the suppressed Community Areas
    print('Working on assigning suppressed trips!')
    for od in df['OD_PAIRS'].unique():

        #select out the trips that are originating from the given community area
        od_trips = df[df['OD_PAIRS'] == od]
        
        #conflate the community area to the census tract centroids that fall within it
        df2 = od_trips[['SUP_PRIVATE_TRIPS', 'SUP_SHARED_TRIPS', 'Pickup Community Area', 'Dropoff Community Area']].merge(conflation[['GEOID','COMMUNITY_AREA']], how = 'left', left_on = 'Pickup Community Area' , right_on = 'COMMUNITY_AREA')
        df2 = df2.merge(conflation[['GEOID','area_num_1']], how = 'left', left_on = 'Dropoff Community Area' , right_on = 'area_num_1', suffixes = ('_PICKUP','_DROPOFF'))
        
        df2['SCALAR'] = np.random.dirichlet(np.ones(len(df2)))
        df2['SCALED_SUP_PRIVATE_TRIPS'] = df2['SUP_PRIVATE_TRIPS']*df2['SCALAR']
        df2['SCALED_SUP_SHARED_TRIPS'] = df2['SUP_SHARED_TRIPS']*df2['SCALAR']
        df3 = df3.append(df2)
        
    df3['TOD'] = tod    
    df_all = df_all.append(df3)
    print(str(df_all.SCALED_SUP_PRIVATE_TRIPS.sum()))
    print(str(df_all.SCALED_SUP_SHARED_TRIPS.sum()))

print('Everyting is Complete!')

Working on tod 1
629505 Trip Records with Suppressed Origin and Destination out of 2523760
186034 Trip Records Outside of Chicago but within cook county out of 2523760
1568392 Trip Records with both trip ends within Chicago out of 2523760
325863 Trip Records with one trip end outside of Cook county out of 2523760
There are 17739.1323649537 Average Weekday Trips that are Suppressed!
Working on assigning suppressed trips!
10325.459399488294
7413.6729654654055
Working on tod 2
86721 Trip Records with Suppressed Origin and Destination out of 289773
18228 Trip Records Outside of Chicago but within cook county out of 289773
153082 Trip Records with both trip ends within Chicago out of 289773
49970 Trip Records with one trip end outside of Cook county out of 289773
There are 5110.291375337681 Average Weekday Trips that are Suppressed!
Working on assigning suppressed trips!
13391.452957236199
9457.97078305518
Working on tod 3
187312 Trip Records with Suppressed Origin and Destination out of 11

In [19]:
df_all.head()

,SUP_PRIVATE_TRIPS,SUP_SHARED_TRIPS,Pickup Community Area,Dropoff Community Area,GEOID_PICKUP,COMMUNITY_AREA,GEOID_DROPOFF,area_num_1,SCALAR,SCALED_SUP_PRIVATE_TRIPS,SCALED_SUP_SHARED_TRIPS,TOD
0,37.604651,17.744186,1.0,1.0,17031830600,1.0,17031830600,1,0.002724,0.102423,0.048330,1
1,37.604651,17.744186,1.0,1.0,17031830600,1.0,17031010702,1,0.009497,0.357113,0.168508,1
2,37.604651,17.744186,1.0,1.0,17031830600,1.0,17031010701,1,0.007772,0.292273,0.137913,1
3,37.604651,17.744186,1.0,1.0,17031830600,1.0,17031010600,1,0.000088,0.003302,0.001558,1
4,37.604651,17.744186,1.0,1.0,17031830600,1.0,17031010503,1,0.001208,0.045443,0.021443,1


In [20]:
df_all[df_all['GEOID_DROPOFF'] == 17031081403]

,SUP_PRIVATE_TRIPS,SUP_SHARED_TRIPS,Pickup Community Area,Dropoff Community Area,GEOID_PICKUP,COMMUNITY_AREA,GEOID_DROPOFF,area_num_1,SCALAR,SCALED_SUP_PRIVATE_TRIPS,SCALED_SUP_SHARED_TRIPS,TOD
9,6.883721,3.55814,1.0,8.0,17031830600,1.0,17031081403,8,0.001040,0.007159,0.003700,1
30,6.883721,3.55814,1.0,8.0,17031010702,1.0,17031081403,8,0.006715,0.046227,0.023894,1
51,6.883721,3.55814,1.0,8.0,17031010701,1.0,17031081403,8,0.000600,0.004129,0.002134,1
72,6.883721,3.55814,1.0,8.0,17031010600,1.0,17031081403,8,0.003222,0.022183,0.011466,1
93,6.883721,3.55814,1.0,8.0,17031010503,1.0,17031081403,8,0.007444,0.051241,0.026486,1
...,...,...,...,...,...,...,...,...,...,...,...,...
261,3.395349,1.27907,77.0,8.0,17031030200,77.0,17031081403,8,0.016934,0.057498,0.021660,8
282,3.395349,1.27907,77.0,8.0,17031030104,77.0,17031081403,8,0.001405,0.004772,0.001798,8
303,3.395349,1.27907,77.0,8.0,17031030103,77.0,17031081403,8,0.000551,0.001872,0.000705,8
324,3.395349,1.27907,77.0,8.0,17031030102,77.0,17031081403,8,0.002067,0.007020,0.002644,8


In [21]:
df_all.SCALED_SUP_PRIVATE_TRIPS.sum() + df_all.SCALED_SUP_SHARED_TRIPS.sum()

72159.92200463495

In [22]:
agg = {'SCALED_SUP_PRIVATE_TRIPS':'sum', 'SCALED_SUP_SHARED_TRIPS':'sum', 'Pickup Community Area':'first', 'Dropoff Community Area':'first'}

In [23]:
grouped = df_all[['SCALED_SUP_PRIVATE_TRIPS','SCALED_SUP_SHARED_TRIPS', 'GEOID_PICKUP','GEOID_DROPOFF', 'Pickup Community Area', 'Dropoff Community Area', 'TOD']].groupby(by = ['GEOID_PICKUP', 'GEOID_DROPOFF','TOD'], as_index = False).agg(agg)

In [24]:
grouped.to_csv('C:/Workspace/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/2019 Suppressed Trips.csv')

In [25]:
grouped['TOTAL_SUP_TRIPS'] = grouped.SCALED_SUP_PRIVATE_TRIPS + grouped.SCALED_SUP_SHARED_TRIPS

In [26]:
origin = grouped[['GEOID_PICKUP', 'TOTAL_SUP_TRIPS']].groupby(by = 'GEOID_PICKUP', as_index = False).sum()
dest = grouped[['GEOID_DROPOFF', 'TOTAL_SUP_TRIPS']].groupby(by = 'GEOID_DROPOFF', as_index = False).sum()

In [27]:
geo = gp.read_file('C:/Workspace/TNC-Demand-Model/Inputs/Census Shapefiles/Chicago Tracts/geo_export_558aad9f-98d8-4dd5-a6b1-c1730155d596.shp')

In [28]:
origin[origin['GEOID_PICKUP'] == 17031081403]

,GEOID_PICKUP,TOTAL_SUP_TRIPS
137,17031081403,106.594792


In [29]:
origin['GEOID_PICKUP'] = origin.GEOID_PICKUP.astype(float)
dest['GEOID_DROPOFF'] = dest.GEOID_DROPOFF.astype(float)

In [30]:
geo['geoid10'] = geo.geoid10.astype(float)

In [31]:
centroids = pd.read_csv('C:/Workspace/TNC-Demand-Model/otp/points.csv')
centroids = gp.GeoDataFrame(centroids)

In [32]:
geo[geo['geoid10'] == 17031081403]

,commarea,commarea_n,countyfp10,geoid10,name10,namelsad10,notes,statefp10,tractce10,geometry
165,8,8.0,031,1.703108e+10,814.03,Census Tract 814.03,Small unpopulated area in CA 32,17,081403,"POLYGON ((-87.60953 41.89096, -87.60484 41.891..."


In [33]:
m = folium.Map([41.8781, -87.6298], zoom_start=11)
    
    
# Add the color for the chloropleth:
folium.Choropleth(
 geo_data=geo,
 name= "Suppressed Pickups",
 data=origin,
 columns = ['GEOID_PICKUP', 'TOTAL_SUP_TRIPS'],
 key_on='feature.properties.geoid10',
 fill_color='BuGn',
 fill_opacity=0.6,
 line_opacity=0.2,
 legend_name='Average Weekday Pickups',
 highlight = True
).add_to(m)

folium.Choropleth(
 geo_data=geo,
 name= "Suppressed Dropoffs",
 data=dest,
 columns = ['GEOID_DROPOFF', 'TOTAL_SUP_TRIPS'],
 key_on='feature.properties.geoid10',
 fill_color='BuGn',
 fill_opacity=0.6,
 line_opacity=0.2,
 legend_name='Average Weekday Dropoffs',
 highlight = True
).add_to(m)

feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)


for tract2 in centroids.GEOID:
    row = centroids[centroids['GEOID'] == tract2]
    folium.CircleMarker([row['Y'], row['X']], popup = str(int(row['GEOID'].values[0])), radius = 1, fill = True, fill_color = 'grey', color = 'grey').add_to(feature_group)

m.add_child(feature_group)


folium.LayerControl().add_to(m)

m.save('C:/Workspace/TNC-Demand-Model/Data Exploration/Suppressed Ridehailing Maps/Suppressed_Trips.html')

print('Everything is Complete!')

Everything is Complete!
